<a href="https://colab.research.google.com/github/eduzortea/SmartPill_Alura_Aula_4/blob/main/SmartPill.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalar SDK

In [ ]:
pip install -q -U google-generativeai

In [ ]:
!pip install virtualenv


In [ ]:
!virtualenv biblioteca1

In [ ]:
!source biblioteca1/bin/activate


biblioteca de autenticacao a servicos do google:

In [ ]:
pip install --upgrade google-auth==2.27.0


In [ ]:
pip install reportlab

In [ ]:
pip install qrcode

Inicializar a IA


In [118]:
from reportlab.lib.pagesizes import letter, landscape
from reportlab.pdfgen import canvas
from reportlab.platypus import Table, TableStyle
from reportlab.lib import colors
from datetime import datetime, timedelta
import qrcode
from google.colab import files
import google.generativeai as genai
from google.colab import userdata

# Obter a chave de API e configurar o GenAI
api_key = userdata.get("SECRET_KEY")
genai.configure(api_key=api_key)

# Instruções do sistema para o GenAI
system_instruction = "Gere uma tabela de programação de medicamentos considerando as informações fornecidas. Cada linha na tabela deve representar um medicamento com as colunas correspondendo aos dias de tratamento, começando pela data de início especificada (ATENÇÃO QUE UM MEDICAMENTO PODE COMEÇAR A MINISTRAÇÃO DEPOIS DA DATA DE OUTRO MEDICAMENTO). Para cada medicamento, ajuste os horários de dose para coincidir com as refeições sempre que possível e não inclua horários de refeição já passados no dia de início."

# Nome do modelo do GenAI
model = genai.GenerativeModel(model_name='gemini-1.5-pro-latest', system_instruction=system_instruction)

# Solicitar informações do usuário
quantidade = int(input("Quantos medicamentos deseja incluir no plano: "))

medicamentos = []
frequencias = []
refeicoes_lista = []
dias_lista = []
inicios_lista = []

for i in range(quantidade):
    medicamentos.append(input(f"Nome do {i + 1}º Medicamento: "))
    frequencias.append(input(f"Frequência (em horas) do {i + 1}º Medicamento: "))
    refeicoes_lista.append(input(f"Com refeições (Sim ou Não) do {i + 1}º Medicamento: "))
    dias_lista.append(input(f"Dias de Tratamento do {i + 1}º Medicamento: "))
    inicios_lista.append(input(f"Data de Início (no formato DD-MM-AAAA) do {i + 1}º Medicamento: "))

# Gerar a programação usando o GenAI
prompt_parts = []

for i in range(quantidade):
    prompt_parts.append(f"Nome do Medicamento: {medicamentos[i]}")
    prompt_parts.append(f"Frequência: {frequencias[i]}")
    prompt_parts.append(f"Com Refeições: {refeicoes_lista[i]}")
    prompt_parts.append(f"Dias: {dias_lista[i]}")
    prompt_parts.append(f"Início: {inicios_lista[i]}")

response = model.generate_content(prompt_parts)

print(response.text)
# Criação do PDF no modo paisagem
pdf_file_name = "medication_schedule.pdf"
c = canvas.Canvas(pdf_file_name, pagesize=landscape(letter))

# Processamento de dados para a tabela a partir da resposta
# Este passo requer que a resposta do texto seja ajustada a um formato de lista de listas adequado para a Table.
# Aqui estamos assumindo que a resposta está formatada como uma série de linhas de texto, cada uma representando um medicamento com suas datas e horários.
data = []
response_lines = response.text.split("\n")
for line in response_lines:
    columns = line.split("|")
    if columns and len(columns) > 1:  # Ignorar linhas vazias ou inválidas
        data.append([col.strip() for col in columns if col.strip()])  # Limpar e remover colunas vazias

# Configuração da tabela
t = Table(data)
t.setStyle(TableStyle([
    ('BACKGROUND', (0,0), (-1,0), colors.grey),
    ('GRID', (0,0), (-1,-1), 1, colors.black),
    ('TEXTCOLOR', (0,0), (-1,0), colors.whitesmoke),
    ('ALIGN', (0,0), (-1,-1), 'CENTER'),
    ('VALIGN', (0,0), (-1,-1), 'MIDDLE')
]))

# Desenho da tabela no canvas
t.wrapOn(c, 800, 300)  # Ajustar as dimensões conforme necessário
t.drawOn(c, 50, 150)  # Ajustar a posição conforme necessário

# Finalizar e salvar o PDF
c.save()

# Fazer o download do arquivo PDF no Colab
files.download(pdf_file_name)

Quantos medicamentos deseja incluir no plano: 2
Nome do 1º Medicamento: paracetamol
Frequência (em horas) do 1º Medicamento: 4h
Com refeições (Sim ou Não) do 1º Medicamento: nao
Dias de Tratamento do 1º Medicamento: 5
Data de Início (no formato DD-MM-AAAA) do 1º Medicamento: 09/05/2024
Nome do 2º Medicamento: amoxilina
Frequência (em horas) do 2º Medicamento: 12h
Com refeições (Sim ou Não) do 2º Medicamento: sim
Dias de Tratamento do 2º Medicamento: 7
Data de Início (no formato DD-MM-AAAA) do 2º Medicamento: 10/05/2024
## Programação de Medicação

| Data | Horário | paracetamol | amoxilina | Refeição |
|---|---|---|---|---|
| 09/05/2024 | 12:00 | X |  |  |
| 09/05/2024 | 16:00 | X |  |  |
| 09/05/2024 | 20:00 | X |  |  |
| 10/05/2024 | 00:00 | X |  |  |
| 10/05/2024 | 04:00 | X |  |  |
| 10/05/2024 | 08:00 | X | X | Café da manhã |
| 10/05/2024 | 12:00 | X |  |  |
| 10/05/2024 | 16:00 | X |  |  |
| 10/05/2024 | 20:00 | X | X | Jantar |
| 11/05/2024 | 00:00 |  | X |  |
| 11/05/2024 | 04

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>